![Welcome IO Quantum 101 Image](../images/WelcomeIOQuantum101Image.png)


# **Lab 03 – Quantum Query Algorithms: The Deutsch–Jozsa Algorithm**

<div style="font-family: 'Arial'; font-size: 16px; line-height: 1.6; text-align: justify;">  
It’s great to see you back again at the IO Quantum Summer School in the IO Scholar Community 🙋🏻‍♂️.
<br>
  
It’s wonderful that we’ve equipped ourselves with all the essentials: from installing the necessary packages, exploring superposition and entanglement, to hands-on practice with both single-qubit and multi-qubit systems. We’ve also witnessed how quantum entanglement can be applied in practice through Quantum Teleportation. With this foundation in place, we are now ready to embark on the journey of exploring quantum algorithms – where the true quantum advantage begins to emerge in tackling challenges once thought intractable for classical computers.

</div>

## **A friendly little reminder**

<div style="font-family: 'Arial'; font-size: 16px; line-height: 1.6; text-align: justify;">   

Just as in the previous lab—and in all the labs to come—we’ll be working with Jupyter notebooks. Code snippets will be provided for you, except in certain cells where your task will be to complete the code to ensure our quantum program runs smoothly. These cells will be clearly marked with a comment like: <span style="font-family: monospace; font-weight: bold; color: #111; background-color: #fff8dc; padding: 2px 6px; border-radius: 4px;"> ### WRITE YOUR CODE BELOW THIS CELL ### </span>.

Another friendly reminder: we’ll be running our quantum programs exclusively on simulators. No worries—these work perfectly well for our purposes.

**Lab 3** focuses on the very first and also the simplest quantum algorithm – **Deutsch Jozsa algorithm**. It is considered a foundational stepping stone for exploring more advanced quantum algorithms. We will begin by revisiting the problem that this algorithm is designed to solve, then construct our own examples of a balanced function and a constant function, and finally apply the algorithm to uncover the answer to the problem.

</div>

## **Pre-checking and Imports**


<div style="font-family: 'Arial'; font-size: 16px; line-height: 1.6; text-align: justify;">   
You should check your Qiskit version before starting the lab. It is recommended to use Qiskit version 2.0 or higher for the best experience.
</div>

In [ ]:
import qiskit
print(f"Qiskit version: {qiskit.__version__}")

In [ ]:
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.visualization import plot_histogram
import random as rd

## **Problem Reminder**

<div style="font-family: 'Arial'; font-size: 16px; line-height: 1.6; text-align: justify;">   

We are given a mysterious function $f(x)$ that takes as input either a single bit or a string of binary bits and returns only one of two possible values: $0$ or $1$.
This mysterious function can be either constant or balanced:

- **Constant function**: always outputs $0$ or always outputs $1$, regardless of the input.

- **Balanced function**: where it always outputs exactly $2^{n-1}$ times $1$ and $2^{n-1}$ times $0$ for inputs of length $n$.
  
**Our goal** is to determine whether the mysterious function above is **a constant function** or **a balanced function**.

</div>

### **Classical Approach**

<div style="font-family: 'Arial'; font-size: 16px; line-height: 1.6; text-align: justify;">   
  
**In classical computation**, in the worst case, we need to query the hidden function $2^{n-1}+1$ times, where $n$ is the length of the binary input string. To make this clearer, we present the following example:

Suppose our input consists of $3$ bits, which gives us a total of $8$ possible input cases. In the classical approach, we must call the function $2^{3-1}+1 = 5$ times to determine whether it is constant or balanced. Imagine that for the first $4$ function calls, we supply different input bits and obtain the output bits shown in the table below. 

| Input (3 bit) | Output |
|---------------|--------|
| 000           | 0      |
| 001           | 0      |
| 010           | 0      |
| 011           | 0      |
  
At this stage, we still cannot conclude whether the hidden function is constant or balanced. Therefore, one more function call is required to determine the answer with certainty. If the next input also returns $0$, then it is clearly a constant function; however, if it returns $1$, then we know for sure that it is a balanced function.

And similarly, **in the best case**, two queries to the oracle can determine if the hidden Boolean function, $f(x)$, is balanced: e.g. if we get both $f(0,0,0,...)\rightarrow 0$ and $f(1,0,0,...) \rightarrow 1$, then we know the function is balanced as we have obtained the two different outputs.  
</div>

### **Modeling the Problem with the Quantum Approach**

<div style="font-family: 'Arial'; font-size: 16px; line-height: 1.6; text-align: justify;">   

With the quantum approach, we only need to query the hidden function $f$ **once** to achieve our goal. Similar to the classical case, we also need to construct both a balanced and a constant version of the function within the quantum circuit, which we will be able to distinguish with just a single query!

The idea is straightforward: we use $n$ qubits as the input, along with one additional qubit representing the function’s output—often referred to as the output qubit. Therefore, our quantum circuit consists of a total of $n+1$ qubits.

<img src="../images/circuitdeu.png" alt="VS Code install prompt example" style="display: block; margin-left: auto; margin-right: auto; width: 45%; border: 1px solid #ccc; border-radius: 8px;">

**For the constant function**, we can design it as follows: for any input, either leave the output unchanged (meaning no operation is applied to the output qubit), or apply an $X$ (NOT) gate to the output qubit for every input. In both cases, the result is always the same (either always $0$ or always $1$) regardless of the input, which clearly captures the idea of a constant function.

**For the balanced function**, we must ensure its defining property: for all possible input cases, the outputs consist of exactly $50\%$ zeros and $50\%$ ones. To achieve this, we can simply apply CNOT gates, where the input qubits act as the control qubits and the output qubit is the target. Additionally, to increase the diversity in the ordering of the outputs, we may randomly apply $X$ gates on the input qubits and then cancel them after the balanced function is applied.

To make this more concrete, let us consider a simple example with $2$ input qubits and $1$ output qubit, as illustrated in the table below.

| Input (2 Qubits)| Output |
|-----------------|--------|
| 00              | 0      |
| 01              | 1      |
| 10              | 1      |
| 11              | 0      |

To increase the diversity in the ordering of the outputs, we apply **an $X$ gate to the first qubit**.

| Input (2 Qubits)| X-gate | Output |   
|-----------------|--------| ------ |   
| 00              | 10     | 1      |   
| 01              | 11     | 0      |   
| 10              | 00     | 0      |   
| 11              | 01     | 1      |   

</div>

## **Designing Constant and Balanced Functions**

In [ ]:
# Number of input qubits
n_input = 5
# Declare quantum and classical registers for inputs, outputs and results
qreg_input = QuantumRegister(n_input, name="input")
qreg_output = QuantumRegister(1, name="output")
creg = ClassicalRegister(n_input, name="result")

### **Constant Oracle**

In [ ]:
# Randomization for more diverse output behavior
output = rd.choice([0, 1])

constant_oracle = QuantumCircuit(qreg_input, qreg_output, creg)

# Add a barrier to make the circuit easier to visualize
constant_oracle.barrier()

if output == 1:
    constant_oracle.x(qreg_output)
constant_oracle.barrier()

constant_oracle.draw('mpl')

### **Balanced Oracle**


In [ ]:
# Generate a random qubit string; qubits with value 1 will be assigned an X gate
# The order from left to right corresponds to the output, input n−1, input n−2, input n−3, and so on.
qstr = format(rd.randint(1, 2**n_input - 1) , '0' + str(n_input) + 'b') 

<div style="font-family: 'Arial'; font-size: 16px; line-height: 1.6; text-align: justify; color: #111; background-color: #fff8dc; padding: 15px; border-radius: 8px;">   

**Exercise 1: Designing a Balanced Oracle**
  
Your task is to design a quantum circuit representing a Balanced Function, which ensures that the outputs always contain an equal number of 0s and 1s. At the same time, maintain diversity in the ordering of the output. Follow these steps:
  
1. For a randomly generated qubit string, apply X gates to the qubits marked as 1 in the string.
2. Apply CNOT gates using the input qubits as control qubits and the output qubit as the target.
3. Undo the X gates added in step 1 to preserve the correctness of the function.
</div>

In [ ]:
balanced_oracle = QuantumCircuit(qreg_input, qreg_output, creg)

balanced_oracle.barrier()

### WRITE YOUR CODE BELOW THIS CELL ###

# Loop through the elements of the qubit string

    # Apply an X gate if the current qubit is 1

        # Perform the X gate

# Loop over the input qubits

    # Apply a CNOT gate with the input qubit as control and the output qubit as target

# Add a barrier to make the circuit easier to visualize
balanced_oracle.barrier()
# Loop through the elements of the qubit string

    # Apply an X gate if the current qubit is 1

        # Apply X gates to reset the input qubits to their original state

### YOUR CODE FINISHES HERE ###
balanced_oracle.barrier()

balanced_oracle.draw('mpl')

## **The Deutsch–Jozsa Algorithm**

<div style="font-family: 'Arial'; font-size: 16px; line-height: 1.6; text-align: justify;">   
 
We assume that the function fed into the quantum circuit of the algorithm is **unknown**; it could be either a balanced function or a constant function, and the choice of which function to use is made **randomly**.
</div>

In [ ]:
# Assume that the function selection is performed randomly
oracle = rd.choice(["balanced", "constant"])

<div style="font-family: 'Arial'; font-size: 16px; line-height: 1.6; text-align: justify; color: #111; background-color: #fff8dc; padding: 15px; border-radius: 8px;">   

**Exercise 2: Designing the Algorithm**
  
Your task is to **design an algorithm** to **determine** whether the function fed into the quantum circuit via the random function above is constant or balanced *(keep everything secret!)*. Follow these steps to construct the algorithm:

1. Create a quantum circuit using the existing registers.
2. Prepare the $\ket -$ state by applying an X gate to the first qubit.
3. Put all input qubits into a quantum superposition.
4. Apply the oracle function (the hidden function) to the circuit.
5. Apply Hadamard gates to all input qubits after the oracle has been applied.
6. Apply measurement gates to all input qubits.

</div>

In [ ]:
dj_circuit = QuantumCircuit(qreg_input, qreg_output, creg)

### WRITE YOUR CODE BELOW THIS CELL ###
# Apply an X gate to the output qubit

# Apply a Hadamard gate to the output qubit to prepare the |−⟩ state

# Add a barrier to make the circuit easier to visualize
dj_circuit.barrier()
# Loop over the input qubits

    # Apply Hadamard gates to all input qubits

# Apply the hidden (oracle) function to the quantum circuit
if oracle=="balanced":
    dj_circuit = dj_circuit.compose(balanced_oracle)
elif oracle=="constant":
    dj_circuit = dj_circuit.compose(constant_oracle)
# Loop over all input qubits again

    # Apply Hadamard gates to all input qubits after applying the function

# Add a barrier to make the circuit easier to visualize.
dj_circuit.barrier()  
# Loop over all input qubits

    # Apply measurement gates
    
### YOUR CODE FINISHES HERE ###

dj_circuit.draw('mpl')

<div style="font-family: 'Arial'; font-size: 16px; line-height: 1.6; text-align: justify;">   
 
Great! After completing the steps above, we can see our quantum circuit fully constructed. By visually inspecting the simulated circuit, you might already get a sense of what kind of function it represents!
  
However, suppose the function is still hidden inside a **“black box.”** To determine its nature, we measure the classical outputs of the input qubits. As expected:
- If the function is constant, all measurements return $0$.
- If the function is balanced, all measurements return $1$.
</div>

<div style="font-family: 'Arial'; font-size: 16px; line-height: 1.6; text-align: justify; color: #111; background-color: #fff8dc; padding: 15px; border-radius: 8px;">   

**Exercise 3: Verifying the Algorithm**
  
Your task is to run the quantum algorithm on a simulator and read the results. Based on the measurements, determine whether the function is balanced or constant.
</div>

In [ ]:
### WRITE YOUR CODE BELOW THIS CELL ###

# Initialize the quantum backend simulator for GHZ state execution
backend_simulator = 
# Compile the GHZ quantum circuit to run on the simulation backend
transpiled_circ =
# Execute the transpiled GHZ circuit on the simulator with 1024 shots
job =
# Retrieve the result of the execution job
result = 
# Count the occurrences of each measurement outcome using the get_counts() method
counts = 
### YOUR CODE FINISHES HERE ###

plot_histogram(counts)

## **Congratulations!**


<div style="font-family: 'Arial'; font-size: 16px; line-height: 1.6; text-align: justify;">   

Congratulations! We have successfully completed Lab 3 of the **IO Quantum Summer School 2025.**

In this lab, we designed and gained a deep understanding of a quantum algorithm known as the **Deutsch-Jozsa algorithm**. Although this algorithm is relatively simple and has limited practical applications, it serves as an important demonstration of **quantum advantage** for certain classes of problems that are challenging for classical computation.

In the next lab, we will continue exploring a quantum algorithm used for **searching** within **unstructured data**: **Grover’s algorithm.**

</div>

## **Additional information**




<div style="font-family: 'Arial'; font-size: 16px; line-height: 1.6; text-align: justify;">   

**Created by**: An Phan
  
**Advised by**: Hoa Nguyen
</div>